In [38]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

In [33]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
#         char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

def createMatrices(sentences, word2Idx, label2Idx, case2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
#         charIndices = []
        labelIndices = []
        
        for word,label in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
#             charIdx = []
#             for x in char:
#                 charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
#             charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, labelIndices]) 
        
    return dataset

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def iterate_minibatches(dataset,batch_len): 
    start = 0
    for i in batch_len:
        tokens = []
        caseing = []
#         char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,c,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            caseing.append(c)
#             char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(caseing)

In [3]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences
# print(sentences[0])
            
        

In [7]:
temp = get_sentences('/srv/Resources/SWE/train_corpus.tsv')
testSentences = get_sentences('/srv/Resources/SWE/test_corpus.tsv')

trainSentences = temp[:6000]
devSentences = temp[6000:]

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


6000
885
1780


In [8]:
print(trainSentences[0])

[['I', '0'], ['längden', '0'], ['torde', '0'], ['få', '0'], ['moderna', '0'], ['par', '0'], ['överleva', '0'], ['om', '0'], ['en', '0'], ['part', '0'], ['bara', '0'], ['lever', '0'], ['i', '0'], ['den', '0'], ['lilla', '0'], ['\xadvärlden', '0'], ['och', '0'], ['den', '0'], ['andra', '0'], ['i', '0'], ['den', '0'], ['stora', '0'], ['.', '0']]


In [12]:
# trainSentences = addCharInformatioin(train_sentences)
# devSentences = addCharInformatioin(dev_sentences)
# testSentences = addCharInformatioin(test_sentences)


In [9]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,label in sentence:
            labelSet.add(label)
            words[token] = True

In [10]:
print(labelSet)

{'PER', 'MISC', 'ORG', 'LOC', '0'}


In [11]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [12]:
print(label2Idx)

{'PER': 0, 'MISC': 1, 'ORG': 2, 'LOC': 3, '0': 4}


In [13]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [21]:
string_words = ' '.join(words.keys())
print(string_words)

grips klartecken artonåring alkoholiserad fantasteri silvergrå förespråkat riktiga improviseras varje vållande Anderssons hyreshus 16.30-tiden 13-åring licenserna Borg 557 fastställa Konfidensindikatorn gallery Dagligen tillsynstillfället Brodys tränarbyten besvärligare tillför Byggandet reservationer objekt tidplanen Superstjärnan fantasi banor arbetsförhållanden startförbud Kirchgasser förskoleåldern kartan Räntan 146.000 makalös visit Morten fåtöljen praktikplatser spretiga modekommentarer omslagspapper Golfaren fara ansvar undre 23-åriga vandrade Söderling Travet dämpade 2:30 kongress budgetproposition bokstavligt 21 vankas hona hjälte distans längtan detaljerade Största Malmqvist styrelsen bordet konstmässorna förbundet önskvärt unik Republikens svenskakunskaper beslutsfattare buffert vittnen följts passa Aktiviteterna snygg sådant anhölls tillfriskna Representation fordonsprogrammet sharialagar vindtät sämsta fortkörningar stundom bevittnade sålde finn förälder På Viberg toppmöte

In [14]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 4, 'initialUpper': 3, 'contains_digit': 6, 'mainly_numeric': 5, 'PADDING_TOKEN': 7, 'allUpper': 2, 'allLower': 1, 'numeric': 0}


In [15]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('jindal/NER-Bi-LSTM-CNN/swedish_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [16]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'engelsman': 8499, 'delat': 19792, 'hamburgerrestaurang': 5656, 'datorspel': 12667, 'mil': 21025, 'DC': 24050, 'röntgenutrustning': 24238, 'Lundin': 13701, 'Arkitekturmuseet': 24870, 'Petrén': 7576, 'bensindunk': 17450, 'Narkolepsiföreningen': 6661, 'förstå': 3715, 'förvåning': 7573, 'ränte\xadpapper': 13865, 'Volt': 25600, 'Villaägarnas': 8982, 'skärpning': 8274, 'affärsmässiga': 22105, 'Närmast': 23139, 'flikar': 11199, 'marknadsaktörer': 21520, 'medicin': 1115, 'Liege': 15821, 'militärer': 13522, 'Alverna': 18596, 'påslagen': 17019, 'turligt': 6580, 'gjordes': 139, 'justitiedepartementet': 951, 'våldsam': 12500, 'Vänsterns': 4732, 'själv-radikaliserad': 8590, 'eldade': 9200, 'skakade': 9786, 'idén': 9514, 'röster': 15408, 'batterierna': 20447, 'Örnroth': 12458, 'betyg': 17693, 'förvärrades': 11407, 'kontrollbehov': 9044, 'trouper': 22120, 'oemotståndligt': 4954, 'milj': 6527, 'inkomstskatter': 22204, 'Bashar': 22778, 'putslustigheter': 7153, 'mentalt': 16751, 'Statskontoret': 1

In [17]:
# characters=set()
# for word in words.keys():
#     for char in word:
#         characters.add(char)
# # print(characters)
# characters = set(characters)
# char2Idx={}
# for char in characters:
#     char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [18]:
print(trainSentences[0]) 

[['I', '0'], ['längden', '0'], ['torde', '0'], ['få', '0'], ['moderna', '0'], ['par', '0'], ['överleva', '0'], ['om', '0'], ['en', '0'], ['part', '0'], ['bara', '0'], ['lever', '0'], ['i', '0'], ['den', '0'], ['lilla', '0'], ['\xadvärlden', '0'], ['och', '0'], ['den', '0'], ['andra', '0'], ['i', '0'], ['den', '0'], ['stora', '0'], ['.', '0']]


In [22]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx)
dev_set = createMatrices(devSentences,word2Idx, label2Idx, case2Idx)
test_set = createMatrices(testSentences, word2Idx, label2Idx, case2Idx)

In [23]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[5385, 21452, 8078, 24908, 14254, 9290, 1855, 17193, 9169, 1579, 3044, 2909, 403, 19288, 2892, 6423, 11633, 19288, 23332, 403, 19288, 24400, 2725], [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4], [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]
23
23


In [24]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [27]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
output = concatenate([words, casing,])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_input (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 300)    7842900     words_input[0][0]                
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 8)      64          casing_input[0][0]               
__________________________________________________________________________________________________
concatenat

In [31]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing = batch       
        model.train_on_batch([tokens, casing], labels)
        a.update(i)
        print(' ')

# model.fit()

Epoch 0/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 40s 
 
40/65 [=================>............] - ETA: 2s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 1/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 2/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
41/65 [=================>............] - ETA: 1s 


64/65 [============================>.] - ETA: 0s 
Epoch 3/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
36/65 [===============>..............] - ETA: 2s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 4/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
40/65 [=================>............] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 5/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
21/65 [========>.....................] - ETA: 3s 


64/65 [============================>.] - ETA: 0s 
Epoch 6/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 7/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
62/65 [===========================>..] - ETA: 0s 


64/65 [============================>.] - ETA: 0s 
Epoch 8/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 9/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 10/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
41/65 [=================>............] - ETA: 1s 


64/65 [============================>.] - ETA: 0s 
Epoch 11/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 12/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 13/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
19/65 [=======>......................] - ETA: 3s 


...

64/65 [============================>.] - ETA: 0s 
Epoch 38/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 39/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
40/65 [=================>............] - ETA: 1s 
 
53/65 [=======================>......] - ETA: 0s 


64/65 [============================>.] - ETA: 0s 
Epoch 40/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
40/65 [=================>............] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 41/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 42/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
33/65 [==============>...............] - ETA: 2s 


64/65 [============================>.] - ETA: 0s 
Epoch 43/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 44/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 45/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
10/65 [===>..........................] - ETA: 2s 


64/65 [============================>.] - ETA: 0s 
Epoch 46/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 47/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
 5/65 [=>............................] - ETA: 1s 
 
51/65 [======================>.......] - ETA: 0s 


64/65 [============================>.] - ETA: 0s 
Epoch 48/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 49/50
 0/65 [..............................] - ETA: 0s 
 
 
 3/65 [>.............................] - ETA: 1s 
 
64/65 [============================>.] - ETA: 0s 


In [34]:
import collections
# a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
predLabels, correctLabels = tag_dataset(dev_batch)   
correctLabels = np.concatenate(correctLabels).ravel()
counter=collections.Counter(correctLabels)


884/885 [============================>.] - ETA: 0s

In [35]:
print(counter)

Counter({4: 13215, 0: 348, 3: 171, 2: 112, 1: 69})


In [36]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

882/885 [============================>.] - ETA: 0s{0: 'PER', 1: 'MISC', 2: 'ORG', 3: 'LOC', 4: '0'}


In [39]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.719210273511991
0.9837585339561624
0.9837002958604754
[0.93314763 0.28346457 0.58181818 0.80319149 0.9944295 ]


In [40]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

1779/1780 [============================>.] - ETA: 0s0.728187965901259
0.985476686884003
0.9842442506167027
[0.91014235 0.29251701 0.62951807 0.81392557 0.99483683]


In [41]:
model.save('swedish_ner_without_char.h5')